<a href="https://colab.research.google.com/github/ogozcelik/neural-project/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
%tensorflow_version 2.x
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras import backend as K
import datetime, os
from tensorflow.keras import layers
from tensorflow.keras import utils
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from PIL import Image
import h5py
import os, sys
# from pyimagesearch.cancernet import CancerNet
# have your model in another file later

In [38]:
# All the hyperparameters here
VAL_SPLIT = 0.15
BATCH_SIZE = 64
OPT = keras.optimizers.Adam(0.1) # optimizer
EARLY_STOPPING_PATIENCE = 5 

In [ ]:
# read train_ims and test_ims.

# convert to tf arrays
data_arr = np.asarray(train_ims, np.float32)
data_tf = tf.convert_to_tensor(data_np, np.float32)

In [31]:
with h5py.File('/content/drive/MyDrive/EEE443/project/eee443_project_dataset_train.h5', 'r') as f:
  # List all groups
  print("Keys: %s" % f.keys())
  # get the data
  train_cap = np.array(list(f[list(f.keys())[0]]))  # captions
  train_imid = np.array(list(f[list(f.keys())[1]])) # indices of images
  # train_ims = np.array(list(f[list(f.keys())[2]]))  # pretrained feature vector
  # train_url = np.array(list(f[list(f.keys())[3]]))  # urls to images
  word_code = np.array(list(f[list(f.keys())[4]]))  # vocabulary

Keys: <KeysViewHDF5 ['train_cap', 'train_imid', 'train_ims', 'train_url', 'word_code']>


In [44]:
with h5py.File('/content/drive/MyDrive/EEE443/project/eee443_project_dataset_test.h5', 'r') as f:
  # List all groups
  print("Keys: %s" % f.keys())
  # get the data
  test_cap = np.array(list(f[list(f.keys())[0]]))   # captions
  test_imid = np.array(list(f[list(f.keys())[1]]))  # indices of images
  # test_ims = np.array(list(f[list(f.keys())[2]])) # pretrained feature vector
  # test_url = np.array(list(f[list(f.keys())[3]]))   # urls to images

Keys: <KeysViewHDF5 ['test_caps', 'test_imid', 'test_ims', 'test_url']>


In [49]:
# construct X samples - to save space, give train_imid as train data and map in the load function.
train_X = np.array([train_ims[id-1] for id in train_imid])
test_X = np.array([test_ims[id-1] for id in test_imid])
# shuffle train and test sets and train-test split
X_shuffled, Y_shuffled = shuffle(train_X, train_cap, random_state=0)
val = int(X_shuffled.shape[0] * VAL_SPLIT)
X_train, Y_train = X_shuffled[:-val], Y_shuffled[:-val]
X_val, Y_val = X_shuffled[-val:], Y_shuffled[-val:]
X_test, Y_test = shuffle(test_X, test_cap, random_state=0)

In [57]:
@tf.function  # we are not loading from disk now, so no need for tf.data?
def load_img_train(img_id, label):  # train_imid and test_imid keep image ids.
  img = tf.io.read_file(os.path.join(TRAIN_PATH, str(img_id) + '.jpg'))
  img = Image.open("gfg.jpg") # tf.image.decode_png(image, channels=3)
  # tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = img.resize((80, 80))  # tf.image.resize(image, (80,80))
  image_sequence = image.getdata()  # ??
  image_array = np.array(image_sequence)
  image_array = np.reshape(image_array,(80,80,3)) # do we need?
  return image_array, label

In [ ]:
@tf.function
def augment(image, label):
	# perform random horizontal and vertical flips
	image = tf.image.random_flip_up_down(image)
	image = tf.image.random_flip_left_right(image)
	# return the image and the label
	return (image, label)
 # do better data augmentation

In [ ]:
trainDS = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
trainDS = (trainDS
	.shuffle(X_train.shape[0])
	# .map(load_images, num_parallel_calls=AUTOTUNE)
	.map(augment, num_parallel_calls=AUTOTUNE)
	.cache()
	.batch(BATCH_SIZE)
	.prefetch(AUTOTUNE)
)
valDS = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
valDS = (valDS
	.shuffle(X_val.shape[0])
	# .map(load_images, num_parallel_calls=AUTOTUNE)
	.map(augment, num_parallel_calls=AUTOTUNE)
	.cache()
	.batch(BATCH_SIZE)
	.prefetch(AUTOTUNE)
)
testDS = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
testDS = (testDS
	.shuffle(X_test.shape[0])
	# .map(load_images, num_parallel_calls=AUTOTUNE)
	.map(augment, num_parallel_calls=AUTOTUNE)
	.cache()
	.batch(BATCH_SIZE)
	.prefetch(AUTOTUNE)
)

In [21]:
# load the inception resnet model
inc_resnet = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(80,80,3), pooling=None
)

219070464/219055592 [==============================] - 3s 0us/step


In [ ]:
# take until the chosen layer. make it trainable.
hidden_layer = inc_resnet.layers[-1].output
cnn_model = tf.keras.Model(inc_resnet.input, hidden_layer)

In [2]:
! pip install tdqm
from tqdm import tqdm

  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=94210c8d57d3c690f249178b5be02726d8db7b8967999e199a8b69b9e6bcd2db
  Stored in directory: /root/.cache/pip/wheels/c6/f0/d9/9fa5ff78c0f9d5a0a427bbbb4893c283520ddfccb885ea2205
Successfully built tdqm


In [ ]:
# Get unique images
encode_train = (train_ims) # train_ims are all sorted. Save with order ids.

image_dataset = tf.data.Dataset.from_tensor_slices(train_ims) # ??
image_dataset = image_dataset.map(num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE)
  # load_image,

for img in tdqm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf in batch_features:
    path = os.path.join(FEATURES_MAP)
    np.save(path_of_feature, bf.numpy())

In [ ]:
# initialize an early stopping callback to prevent the model from
# overfitting
early_stop = EarlyStopping(
	monitor="val_loss",
	patience=EARLY_STOPPING_PATIENCE,
	restore_best_weights=True)

In [ ]:
# model for visualizing the activations
layer_outputs = [layer.output for layer in inc_resnet.layers[:20]] 
activation_model = keras.models.Model(inputs=inc_resnet.input, outputs=layer_outputs)
activations = activation_model.predict(img) 

In [ ]:
# plot all the activations
layer_names = []
for layer in classifier.layers[:12]:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    
images_per_row = 16
​
for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    # choose until which layer you want to take

In [ ]:
H = model.fit(
	x=trainDS,
	validation_data=valDS,
	class_weight=classWeight,
	epochs=config.NUM_EPOCHS,
	callbacks=[es],
	verbose=1)